https://www.sephora.com/robots.txt

## Set Up

In [96]:
from __future__ import print_function, division

# if needed: pip install requests or conda install requests
import requests

requests.__path__

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import chromedriver_binary

import pandas as pd
import pickle
import json

### gather all links from xml sitemap

In [35]:
xml_url = 'https://www.sephora.com/products-sitemap.xml'

response = requests.get(xml_url)

In [36]:
# response
soup = BeautifulSoup(response.text, "xml")

In [40]:
all_links = []

for _ in soup.find_all('loc'):
    all_links.append(_.text)

print(len(all_links))
all_links[0:5]

10157


['https://www.sephora.com/product/double-ended-blemish-extractor-P0417',
 'https://www.sephora.com/product/body-lava-P04546871',
 'https://www.sephora.com/product/pro-filtr-instant-retouch-primer-P04897543',
 'https://www.sephora.com/product/shalimar-eau-de-toilette-P0771',
 'https://www.sephora.com/product/amarige-P0782']

### gather links from each category

In [46]:
skincare_url = 'https://www.sephora.com/shop/skincare'

response_skincare = requests.get(skincare_url)
soup2 = BeautifulSoup(response_skincare.text,'lxml')

In [50]:
for _ in soup2.find_all(class_='css-oftl0u'):
    print(_['href'])

/shop/acne-treatment-blemish-remover
/shop/anti-aging-skin-care
/shop/dark-spot-remover
/shop/pore-minimizing-products
/shop/dry-skin-treatment
/shop/wrinkle-cream-wrinkle-remover
/shop/skin-brighteners-dull-skin-treatments
/shop/moisturizer-skincare
/shop/night-cream
/shop/cleansing-oil-face-oil
/shop/face-mist-face-spray
/shop/neck-cream-decollete
/shop/bb-cream-cc-cream
/shop/face-wash-facial-cleanser
/shop/exfoliating-scrub-exfoliator
/shop/makeup-remover-skincare
/shop/face-wipes
/shop/facial-toner-skin-toner
/shop/blotting-paper-oil-control-skincare
/shop/face-serum
/shop/acne-products-acne-cream
/shop/facial-peels
/shop/vitamins-for-hair-skin-nails
/shop/holistic-wellness
/shop/wellness-tools-rollers
/shop/eye-cream-dark-circles
/shop/eye-mask
/shop/facial-treatment-masks
/shop/sheet-masks
/shop/facial-cleansing-brushes
/shop/hair-removal-products
/shop/anti-aging-tools
/shop/teeth-whitening-at-home
/shop/face-sunscreen
/shop/sunblock
/shop/after-sun-care
/shop/face-tanner-self-

### keeping categories: moisturizers, cleansers, treatments, eye care, masks, sun care, lip treatments

/shop/moisturizer-skincare
/shop/night-cream
/shop/cleansing-oil-face-oil
/shop/face-mist-face-spray
/shop/neck-cream-decollete
/shop/bb-cream-cc-cream

/shop/face-wash-facial-cleanser
/shop/exfoliating-scrub-exfoliator
/shop/makeup-remover-skincare
/shop/face-wipes
/shop/facial-toner-skin-toner
/shop/blotting-paper-oil-control-skincare

/shop/face-serum
/shop/acne-products-acne-cream
/shop/facial-peels

/shop/eye-cream-dark-circles
/shop/eye-mask

/shop/facial-treatment-masks
/shop/sheet-masks

/shop/face-sunscreen
/shop/sunblock

/shop/lip-balm-lip-care
/shop/lip-sunscreen

### tier 2 priorities: wellness,  high-tech tools

/shop/vitamins-for-hair-skin-nails
/shop/holistic-wellness
/shop/wellness-tools-rollers

/shop/facial-cleansing-brushes
/shop/hair-removal-products
/shop/anti-aging-tools
/shop/teeth-whitening-at-home

### excluding categories: self tanner

/shop/after-sun-care
/shop/face-tanner-self-tanner-face
/shop/self-tanner-self-tanning


In [58]:
cat_urls = ['/shop/moisturizer-skincare',
            '/shop/night-cream',
            '/shop/cleansing-oil-face-oil',
            '/shop/face-mist-face-spray',
            '/shop/neck-cream-decollete',
            '/shop/bb-cream-cc-cream',
            '/shop/face-wash-facial-cleanser',
            '/shop/exfoliating-scrub-exfoliator',
            '/shop/makeup-remover-skincare',
            '/shop/face-wipes',
            '/shop/facial-toner-skin-toner',
            '/shop/blotting-paper-oil-control-skincare',
            '/shop/face-serum',
            '/shop/acne-products-acne-cream',
            '/shop/facial-peels',
            '/shop/eye-cream-dark-circles',
            '/shop/eye-mask',
            '/shop/facial-treatment-masks',
            '/shop/sheet-masks',
            '/shop/face-sunscreen',
            '/shop/sunblock',
            '/shop/lip-balm-lip-care',
            '/shop/lip-sunscreen'
            ]

### pull IDs for each cat

In [59]:
# url = 'https://www.sephora.com/shop/moisturizer-skincare'
dict_ids = {}

for cat in cat_urls:
    url = 'https://www.sephora.com'+cat

    response_cat = requests.get(url)

    soup_cat = BeautifulSoup(response_cat.text,'lxml')

    ids = []
    for _ in soup_cat.find_all(class_ = 'css-ix8km1'):
        ids.append(_['data-uid'].split()[0])
    dict_ids.update({cat:ids})

len(dict_ids)

In [68]:
#total ids scraped
len(dict_ids)*12

276

In [78]:
#gather unique ids
unique_ids = []

for key,val in dict_ids.items():
    for v in val:
        if v not in unique_ids:
            unique_ids.append(v)

len(unique_ids)

239

In [100]:
# lst = [link for link in all_links if ids in unique_ids]

In [98]:
link_list = []
for link in all_links:
    for i in unique_ids:
        if i == link[link.find('-P')+1:]:
            link_list.append(link)

len(link_list)

239

In [99]:
# save out link_list
with open('pickles/link_list1.pickle', 'wb') as to_write:
    pickle.dump(link_list, to_write)